
# Modélisation

*OPENCLASSROOMS - Parcours Data Analyst V2 - Adeline Le Ray*
<hr>

## Sommaire

* [3. Régressions]
    * [3.1 Régression linéaire simples]
        * [3.1.1. Méthode des moindres carrés ordinaires (MCO)]
 

# 1. Echantillonnage

## 1.1 Echantillonnage stratifié

### Avec Pandas

````
# Création de l'échantillon d'entraînement et de l'échantillon de test en respectant la répartition globale
train_2 = df_reg2.groupby('age').apply(lambda x: x.sample(frac=0.80)).droplevel(0)
index=train_2.reset_index().iloc[:,0].tolist()
test_2 = df_reg2.loc[~df_reg2.index.isin(index)]
````

# 3. Régressions

## 3.1 Régression linéaire 

### 3.1.1. Méthode des moindres carrés ordinaires (MCO)

$\hat{y_{i}}=\hat{a}.x_{i} +\hat{b}$ avec $\hat{a}=\frac{s_{X,Y}}{s_{X}^2}$ et $\hat{b}=\bar{y}-\hat{a}.\bar{x}$

Le traitement statistique, la régression linéaire avec estimation par la méthode des moindres carrés, est peu robuste aux outliers.

###### Code :

- **Calculs des dépenses et de l'attente**

````

import datetime as dt

# Selection du sous-échantillon
courses = operations[operations.categ == "COURSES"]

# On trie les opérations par date
courses = courses.sort_values("date_operation")

# On ramène les montants en positif
courses["montant"] = -courses["montant"]

# calcul de la variable attente
r = []
last_date = dt.datetime.now()
for i,row in courses.iterrows(): #Iterate over DataFrame rows as (index, Series) pairs.
    days = (row["date_operation"]-last_date).days
    if days == 0:
        r.append(r[-1])
    else:
        r.append(days)
    last_date = row["date_operation"]
courses["attente"] = r
courses = courses.iloc[1:,]#df sans la première ligne

# on regroupe les opérations qui ont été effectués à la même date
# (courses réalisées le même jour mais dans 2 magasins différents)
a = courses.groupby("date_operation")["montant"].sum()
b = courses.groupby("date_operation")["attente"].first()
courses = pd.DataFrame({"montant":a, "attente":b})#colonne 'montant'=a et attente = b
````

- **Calcul de $\hat{a}$ et $\hat{b}$**

````
import statsmodels.api as sm
Y = courses['montant']
X = courses[['attente']]
X = X.copy() # On modifiera X, on en crée donc une copie
X['intercept'] = 1.
result = sm.OLS(Y, X).fit() # OLS = Ordinary Least Square (Moindres Carrés Ordinaire)
a,b = result.params['attente'],result.params['intercept']
````

Une régression linéaire prédit une variable en fonction d'une ou plusieurs variables.  `sm.OLS`  s'attend donc à trouver une unique colonne (c.-à-d. un  `pd.Series`  ) en premier argument (ici `Y`), mais s'attend à trouver potentiellement plusieurs colonnes en 2nd argument (ici `X`, qui est un  `pd.DataFrame`  ). Pour sélectionner plusieurs colonnes d'un dataframe, on passe une liste de noms de colonnes. Et comme une liste s'écrit entre crochets, ceux-ci viennent s'ajouter aux crochets déjà présents !


- **Affichage de la régression**
````
plt.plot(courses.attente,courses.montant, "o")
plt.plot(np.arange(15),[a*x+b for x in np.arange(15)])
plt.xlabel("attente")
plt.ylabel("montant")
plt.show()
````

En ligne 2,  np.arange  crée une liste de nombres entiers allant de 0 à 14 :  `[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]`  .

On place cette liste en abscisse. Pour chacune de ces 15 valeurs, on calcule les ordonnées grâce à la formule y=ax+b comme ceci :  `[a*x+b for x in np.arange(15)]`  . On vient donc de créer une série de points tous alignés sur la droite d'équation y=ax+b. La ligne 2 affiche tous ces points, en les reliant entre eux, ce qui nous donne une belle ligne !

### 3.1.2 Coefficient de détermination R²: pourcentage expliqué de la variance

$$SCT=SCE+SCR$$ 

$$\sum\limits _{j=1} ^{n}(y_{j}-\bar y)^2=\sum\limits _{j=1} ^{n}(\hat{y_{j}}-\bar y)^2 + \sum\limits _{j=1} ^{n}(y_{j}-\hat y_{j})^2$$
$SCT$ (somme des carrés totale) traduit la variation totale de Y , $SCE$ (somme des carrés expliquée) traduit la variation expliquée par le modèle et $SCR$ (somme des carrés résiduelle) traduit la variation inexpliquée par le modèle.

Pour la régression linéaire, le pourcentage de variation expliquée est donné par le coefficient de détermination noté $R^2$ :

$$R^2=\frac{SCE}{SCT}$$

####  avec statsmodels.api
```
import statsmodels.api as sm
Y = dt['Position']
X = dt[["Age"]].copy()
X['intercept'] = 1
result = sm.OLS(Y, X).fit()

a,b = result.params
print(a, b, result.rsquared)

```
####  avec sklearn.metrics
````
# Calcul du coefficient de détermination R²
from sklearn.metrics import r2_score 
R_square = r2_score(test_1.montant_total_achat, test_1.estim) 
print("Echantillon de test : Le coefficient de détermination R² de la régression linéaire de", round(R_square,2))
````


#### Interprétation et utilisation

On peut voir le $R^2$  comme l’erreur du modèle divisé par l’erreur d’un modèle basique qui prédit tout le temps la moyenne de la variable à prédire.
- $R^2$ = 1 : modèle parfait
- 0 < $R^2$ <1 : Le score $R^2$  est d’autant plus élevé que le modèle est performant, et vaut au maximum 100%, lorsque toutes les prédictions sont exactes. 
- $R^2$  <0 : Il n’y a pas de score minimum, mais un modèle simple prédisant tout le temps la valeur moyenne atteint un score R2 de 0%. Par conséquent un score R2 négatif signifie que les prédictions sont moins bonnes que si l’on prédisait systématiquement la valeur moyenne.


Le $R^2$  a deux spécificités : 
- Il facilite la comparaison entre différents modèles. Dire qu’un modèle a une MSE de 25 ne permet pas de conclure si le modèle est correct car cela dépend des valeurs prises par la variable à prédire. Alors que la normalisation faite dans le $R^2$  permet de dire qu’un modèle ayant moins de 20% de $R^2$  n’est pas performant et qu’au contraire un modèle qui atteint plus de 80% de $R^2$  est performant.
- Il est en revanche peu interprétable et ne donne pas d’information sur l’erreur moyenne du modèle. En effet, si le $R^2$  permet de comparer la performance du modèle avec une performance basique, il ne permet pas pour autant de dire quelle erreur est faite en moyenne sur les prédictions. Il faut souvent le combiner avec d’autres métriques afin de mieux comprendre la performance du modèle comme la MSE ou la MAE.

````
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X = df_complet.loc[(~df_complet['pop_ss_moyen_alim_saine_(%)'].isna())\
                   &(df_complet['c_income']!=9010),:].iloc[:,10:26]
y = df_complet.loc[(~df_complet['pop_ss_moyen_alim_saine_(%)'].isna())\
                   &(df_complet['c_income']!=9010),'pop_ss_moyen_alim_saine_(%)'].values

# Création de l'échantillon d'entraînement et de l'échantillon de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

model = LinearRegression()

model.fit(X_train,y_train) 

y_train_predict = model.predict(X_train) 

r_squared = r2_score(y_train,y_train_predict)
print(r_squared)


y_test_predict = model.predict(X_test) 

r_squared = r2_score(y_test,y_test_predict)
print(r_squared)
````